<a href="https://colab.research.google.com/github/ujs/cifar10/blob/master/cifar2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dataujs","key":"4070effe6d105acff94a4a8b4d44777f"}'}

In [0]:
!mkdir -p ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c cifar-10

  0% 0.00/655k [00:00<?, ?B/s]
100% 655k/655k [00:00<00:00, 43.0MB/s]
  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 81.5MB/s]
 98% 599M/610M [00:06<00:00, 71.2MB/s]
100% 610M/610M [00:06<00:00, 96.2MB/s]
 79% 83.0M/105M [00:01<00:00, 65.9MB/s]
100% 105M/105M [00:01<00:00, 93.8MB/s] 


In [7]:
!ls

10000.png  17503.png  25004.png  32506.png  40007.png  4750.png
10001.png  17504.png  25005.png  32507.png  40008.png  47510.png
10002.png  17505.png  25006.png  32508.png  40009.png  47511.png
10003.png  17506.png  25007.png  32509.png  4000.png   47512.png
10004.png  17507.png  25008.png  3250.png   40010.png  47513.png
10005.png  17508.png  25009.png  32510.png  40011.png  47514.png
10006.png  17509.png  2500.png	 32511.png  40012.png  47515.png
10007.png  1750.png   25010.png  32512.png  40013.png  47516.png
10008.png  17510.png  25011.png  32513.png  40014.png  47517.png
10009.png  17511.png  25012.png  32514.png  40015.png  47518.png
1000.png   17512.png  25013.png  32515.png  40016.png  47519.png
10010.png  17513.png  25014.png  32516.png  40017.png  4751.png
10011.png  17514.png  25015.png  32517.png  40018.png  47520.png
10012.png  17515.png  25016.png  32518.png  40019.png  47521.png
10013.png  17516.png  25017.png  32519.png  4001.png   47522.png
10014.png  17517.png  25018.

In [5]:
!7z e train.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 109723070 bytes (105 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 557 - train/105.png                          2% 924 - train/1083.png                           2% 1188 - train/11067.png                             2% 1402 - train/1126.png                            2% 1587 - train/11426.png                             2% 

In [6]:
#importing libraries
from random import random
import pandas as pd
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
from matplotlib.image import imread
import keras
from sklearn.model_selection import train_test_split
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras import layers
from keras import backend
from keras import Input
from keras.preprocessing import image

Using TensorFlow backend.


In [8]:
filename = 'trainLabels.csv'
map_csv = pd.read_csv(filename)
map_csv.head()


,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [0]:
# Map categorical labels to integer using dictionaries

def create_label_map(map_csv):
  '''Function to map class labels to integers and vice versa'''
  #extracting set of unique labels, converting to sorted list. In this case there are 10 unique labels
  labels = list(map_csv.label.unique())
  labels.sort()
  #forward map
  labels_map = {labels[i]:i for i in range(len(labels))}
  #Inverse map
  inv_labels_map = {i:labels[i] for i in range(len(labels))}
  return labels_map, inv_labels_map

# Map all image file names to labels. Hash tables will taking constant time for retrieval
def create_file_map(map_csv):
  mapping = dict()

  for i in range(len(map_csv)):
    name, label = map_csv.id.iloc[i], map_csv.label.iloc[i]
    mapping[name] = label
  return mapping

  #calling the funciton
label_dict = create_file_map(map_csv)
label_dict[10]
#Let's call the above function
mapping, inv_mapping = create_label_map(map_csv)



In [0]:
#One hot encoding representation of image label
def one_hot_encode(label, mapping):
  '''First input is label of the given image. Second input is label to integer map dictionary. Output is one hot encoded representation for the image label '''
  encoding = np.zeros(len(mapping), dtype = 'uint8')
  encoding[mapping[label]] = 1
  return encoding
  

In [0]:
import random
# load all images into memory
def load_dataset(folder, label_dict, mapping):
  photos, targets = list(), list()
  # enumerate files in the directory
  filenames = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder,f)) and f[-3:] == 'png']
  random.shuffle(filenames)
  for filename in filenames:
    # load image
    photo = keras.preprocessing.image.load_img(filename, target_size=(32,32))
    # convert to numpy array
    photo = keras.preprocessing.image.img_to_array(photo, dtype='uint8')
    # get labels for each image
    label_num = int(filename.split('.')[0])
    label = label_dict[label_num]
    # one hot encoded represenation of label
    target = one_hot_encode(label, mapping)
    # store
    photos.append(photo)
    targets.append(target)
  X = np.asarray(photos, dtype='uint8')
  y = np.asarray(targets, dtype='uint8')
  return X, y


In [18]:

#testing function
folder = '/content/'
result = load_dataset(folder, label_dict, mapping)
#Saving X,y in a compressed file
np.savez_compressed('cifar.npz',result[0],result[1])


# load prepared planet dataset
from numpy import load
data = load('cifar.npz')
X, y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, y.shape)

Loaded:  (50000, 32, 32, 3) (50000, 10)


In [0]:
#Split data into train, validation, and test set
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, random_state=1)
trainX, valX, trainY, valY = train_test_split(X, y, test_size=0.25, random_state=1)

#using ImageDataGenerator for random sampling and data normalization, augmentation
train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True, vertical_flip=True, rotation_range=90)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(trainX,trainY, batch_size = 128)
val_generator = train_datagen.flow(valX,valY, batch_size = 128)

In [20]:
# Improving the Model (Dropout)
def define_updated_model(input_shape = (32,32,3), output_shape = 10):
  input_tensor = Input(shape = input_shape)
  x = layers.Conv2D(32,(3,3), activation = 'relu', padding = 'same')(input_tensor)
  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = layers.MaxPooling2D((2, 2))(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
  x = layers.MaxPooling2D((2, 2))(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
  x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
  x = layers.MaxPooling2D((2, 2))(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(x)
  x = layers.Dropout(0.5)(x)
  output_tensor = layers.Dense(output_shape, activation='sigmoid')(x)
  model = Model(input_tensor, output_tensor)

  # compilation
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#test function
model1 = define_updated_model(input_shape = (32,32,3), output_shape = 10)
#model_baseline.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
history = model1.fit_generator(train_generator, steps_per_epoch = 1000, epochs = 20, validation_data = val_generator, validation_steps = 500)
model1.save('cifar10_dropout')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





1000/1000 [==============================] - 1066s 1s/step - loss: 2.0316 - acc: 0.2509 - val_loss: 2.0143 - val_acc: 0.2681
Epoch 2/20
1000/1000 [==============================] - 1073s 1s/step - loss: 1.8489 - acc: 0.3253 - val_loss: 1.7288 - val_acc: 0.3624
Epoch 3/20
1000/1000 [==============================] - 1063s 1s/step - loss: 1.7834 - acc: 0.3542 - val_loss: 1.6425 - val_acc: 0.4206
Epoch 4/20
 424/1000 [===========>..................] - ETA: 8:59 - loss: 1.7351 - acc: 0.3740